In [19]:
from urllib.parse import quote
print(quote(";&&|`$("))

%3B%26%26%7C%60%24%28


In [60]:
from urllib.parse import unquote #This used to decode , encode string data

#For big string input
#log_data = ["2025-12-31T05:03:37.934594Z,127.0.0.1:40924,GET,http://192.168.122.170/dvwa/vulnerabilities/xss_r/?name=%3Cscript%3Ealert%28%22hekk%22%29%3C%2Fscript%3E,/dvwa/vulnerabilities/xss_r/?name=%3Cscript%3Ealert%28%22hekk%22%29%3C%2Fscript%3E,XSS,0,0.0,0"]

# Danger signs for Command Injection

cmd_sign = [";", "&&", "||", "|", "`", "$("]

#This used to read List as String so urllib are read only String not List
#raw_text = log_data[0]

raw_text = input("Enter text to check:")


#Decode the text (converts %3B to ;,etc.)
decoded_input = unquote(raw_text)

#Check for attacks in given string
detected = False
for sign in cmd_sign:
    if sign in decoded_input:
        print(f"DANGER: Command Injection character '{sign}' found")
        detected = True
        break

if not detected:
    print(f"No Command Injection detected.")
    print(f"Decoded result: {decoded_input}")

Enter text to check:;ls
DANGER: Command Injection character ';' found


In [ ]:
OTHER ATTACKS:

In [27]:
import re
from urllib.parse import unquote_plus

def security_scanner():
    # settings
    sqli = [" or 1=1", "union select", "sleep(", "benchmark(", "--", "order by"]
    cmd = [";", "&&", "||", "|", "`", "$("]
    path = ["../", "..\\", "php://", "file://", "etc/passwd"]
    xss = ["<script", "javascript:", "onerror=", "onload="]
    nosql = ["$gt", "$ne", "$where", "$regex"]
    
    blocked_ext = [".php", ".py", ".exe", ".jsp", ".sh", "js", ".bash", ".c", ".asm", ".java", ".class"]
    allowed_ext = [".png", ".jpg", ".jpeg", ".gif", ".pdf", ".txt", "text"]

    raw_data = input("enter request: ")
    text = unquote_plus(unquote_plus(raw_data)).lower()
    
    status = "normal"

    if 'filename="' in raw_data:
        matches = re.findall(r'filename="([^"]+)"', raw_data, re.IGNORECASE)
        
        found_file_issue = False
        for fname in matches:
            fname = fname.lower()

            # block dangerous extensions
            for ext in blocked_ext:
                if fname.endswith(ext):
                    status = "file_upload_abuse"
                    found_file_issue = True
                    break

            # block double extensions
            if not found_file_issue and fname.count(".") >= 2:
                status = "file_upload_abuse"
                found_file_issue = True

            # allow safe extensions
            if not found_file_issue:
                safe = False
                for ext in allowed_ext:
                    if fname.endswith(ext):
                        status = "normal"
                        safe = True
                        break
                
                # unknown extension -> block
                if not safe:
                    status = "file_upload_abuse"
                    found_file_issue = True
            
            if found_file_issue:
                break

    # 2. non-file attack checks
    if status == "normal":
        found = []
        if any(s in text for s in sqli): found.append("sql_injection")
        if any(s in text for s in cmd): found.append("command_injection")
        if any(s in text for s in path): found.append("path_traversal")
        if any(s in text for s in xss): found.append("xss")
        if any(s in text for s in nosql): found.append("nosql_injection")
        
        if found:
            status = "|".join(found)

    # final result
    if status != "normal":
        print(f"block: {status}")
    else:
        print("normal")

security_scanner()

enter request: 2026-01-04T10:40:00Z,127.0.0.1,GET,https://google.com/search?q=weather,/search,NORMAL,0,0.0
normal


In [ ]:

SQL:
    2026-01-04T10:00:00Z,127.0.0.1,GET,http://site.com/login?user=admin' or 1=1 --,/login,0,0.0
    
COMMAND:
    2026-01-04T10:05:00Z,127.0.0.1,POST,http://site.com/exec,/exec,; cat /etc/passwd,20,4.5

XSS:
    2026-01-04T10:10:00Z,127.0.0.1,GET,http://site.com/search?q=<script>alert('xss')</script>,/search,0,0.0

FILE UPLOAD:
    2026-01-04T10:15:00Z,127.0.0.1,POST,http://site.com/upload,/upload,Content-Disposition: form-data; name="file"; filename="shell.php",500,7.2
    
FILE UPLOAD DUBLE .png.exe:
    2026-01-04T10:20:00Z,127.0.0.1,POST,http://site.com/upload,/upload,Content-Disposition: form-data; name="file"; filename="image.jpg.exe",1200,6.8
    
PATH TRAVEL /LFI:
    2026-01-04T10:25:00Z,127.0.0.1,GET,http://site.com/view?file=../../../../etc/passwd,/view,0,0.0

NOSQL:
    2026-01-04T10:30:00Z,127.0.0.1,POST,http://site.com/api/login,/api/login,{"user": {"$ne": null}, "pass": {"$gt": ""}},85,5.1

COMBINED ATTACK:
    2026-01-04T10:35:00Z,127.0.0.1,POST,http://site.com/test,/test,user=admin'--&cmd=;ls -la,50,4.9

NORMAL:
    2026-01-04T10:40:00Z,127.0.0.1,GET,https://google.com/search?q=weather,/search,0,0.0
    
NOW WE CAN PROTECT TOTAL 9 ATTACKS NOW DDOS ALSO BUT AS LIMITED ONLY


In [1]:
# 1.SIGNATURES (SQLI, CMD, PATH,XSS,NOSQL,FILE INJECTION)
sqli = [" or 1=1", "union select", "sleep(", "benchmark(", "--", "order by"]
cmd = [";", "&&", "||", "|", "`", "$("]
path = ["../", "..\\", "php://", "file://", "etc/passwd", "boot.ini"]
xss = ["<script", "javascript:", "onerror=", "onload=", "alert("]
nosql = ["$gt", "$ne", "$where", "$regex"]

# Critical Server Attacks
server_attack = [
"{{", "${", "<%=",                # SSTI
"<!entity", "<!doctype",          # XXE
"%0d%0a", "\\r\\n",               # CRLF
"__proto__", "constructor.prot",  # Prototype Pollution
"o:", "a:", "s:", "ac ed 00 05"   # Deserialization (PHP/Java)
]

blocked_ext = [".php", ".py", ".exe", ".jsp", ".sh", "js", ".bash", ".c", ".asm", ".java", ".class"]
allowed_ext = [".png", ".jpg", ".jpeg", ".gif", ".pdf", ".txt", "text"]

enter request: sdsjdjsjkdskjdsd:sdsd
normal (entropy: 2.04)
